### Entegra Report Analysis

In [228]:
import unicodedata
import os
import sys
import re
import numpy as np
import pandas as pd
from pathlib import Path
import nltk
from nltk import word_tokenize 
import shutil
from os.path import isfile, join
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import locale
from locale import atof

##### Functions

In [229]:
# custom character alphabet for word
tr = re.compile(r"[abcçdefgğhıijklmnoöprsştuüvyzqxw]+", re.IGNORECASE|re.UNICODE) # Turkish filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n0123456789'
en = re.compile(r"[abcdefghıijklmnopqrstxuvwyz]+", re.IGNORECASE|re.UNICODE) # English
nl = re.compile(r"[abcdefghıijklmnopqrstxuvwyzāăēĕīĭōŏūŭ]+", re.IGNORECASE|re.UNICODE) # Dutch (Flemenk)
fr = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàâæèéêëîïôœùûüÿ]+", re.IGNORECASE|re.UNICODE) # French
de = re.compile(r"[abcdefghıijklmnopqrstxuvwyzäöüß]+", re.IGNORECASE|re.UNICODE) # German
es = re.compile(r"[abcdefghıijklmnopqrstxuvwyzñáéíóú]+", re.IGNORECASE|re.UNICODE) # Spanish (¿¡)
pt = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàáâãéêíóôõú]+", re.IGNORECASE|re.UNICODE) # Portuguese
it = re.compile(r"[abcdefghıijklmnopqrstxuvwyzàéèìòùî]+", re.IGNORECASE|re.UNICODE) # Italian
ar = re.compile(r"[ٿصؼۤڳڲؿڎػڠجڿ٬ٸؽؒؓطۄڀۂؘؔتٚڛےٝڜؖڦ٫ډ۰زۇٖۀ،لۓعٮێڔ۶ؚۧۜڤۏإٞٷؗۖ؈ژۣؕؑٴأۻڸۺگاڴڹۯ؉ْڌ؍ي؟ـٟړۅؐڶُىڽېًۢؠضۚڄٛڏٱۦ٩س٦ڼڂٔۘ٠ښٌٍ۬ٳ۾ٲږذۋٵٜ٘ڞڅںٗهڣۿپڒۥۗڋیؙم؞ثۨٹڵڪظٶۭ١ڭەڨحٕ؎ٺڷٰ۪۫ڻڥۛڑڟټآڡغګ؊ّٯڧڮ؏ۮ؋ؤ٪ؾڗۼق۟دکوِڰڐۃ۽ہفرڇچڝ۴بۈٽڕۡھةٓڃئ؛ڬٙڙڢڱۊَۆۉځ۠ۍۑۙڊنءڈٻشڍ؇۵كخ\ا]+", re.IGNORECASE|re.UNICODE) # Arabic

In [230]:
def convert_one_character_letter(text):
    '''This function converts two byte occupy of letter to one byte unicode character without any visual change \n
    like as Turkish character ç,ş,ö,ğ. \n
    convert_one_character_letter(text): text is any string word or sentence.
    '''
    new_text = unicodedata.normalize('NFC', f"{text}")
    return new_text

In [231]:
def clean_text(text, custom_alp=tr): # for only string in specific language
    '''This function extract custom_alp character from string \n
    clean_text(text, custom_alp): text is any string word or sentence and custom_alp is specific alphabet.
    '''
    # text_clean = re.findall(custom_alp, text)
    text_result_list = []
    text_list = re.findall(r'\S+', text)
    for text_var in text_list:
        text_clean = re.findall(custom_alp, str(text_var))
        text_result_var = "".join(text_clean)
        text_result_list.append(text_result_var)
    text_result = " ".join(text_result_list)
    return text_result

In [232]:
def lower_func(text):
    '''This function convert string character to lowercase. \n
    lower_func(text=str): text is any string word or sentence
    '''
    string_lower = str.lower(text)
    return string_lower

In [233]:
def upper_func(text):
    '''This function convert string character to uppercase. \n
    upper_func(text=str): text is any string word or sentence
    '''
    string_upper = str.upper(text)
    return string_upper

In [234]:
def capitalize_func(text):
    '''This function convert string character to capitalize case. \n
    capitalize_func(text=str): text is any string word or sentence
    '''
    string_capitalize = str.capitalize(text)
    return string_capitalize

In [235]:
def title_func(text):
    '''This function convert string character to title as each word start with capital letter. \n
    title_func(text=str): text is any string word or sentence
    '''
    string_title = str.title(text)
    return string_title

In [236]:
def character_clean_lower_text(text, custom_alp=tr): # for only string in specific language
    '''This function converts two byte occupy of letter to one byte unicode character without any visual change \n
    like as Turkish character ç,ş,ö,ğ. And also extract custom_alp character from string and convert to lowercase. \n
    character_clean_lower_text(text, custom_alp): text is any string word or sentence and custom_alp is specific alphabet.  
    '''
    new_text = unicodedata.normalize('NFC', f"{text}")
    text_result_list = []
    text_list = re.findall(r'\S+', new_text)  # like as word tokenize
    for text_var in text_list:
        text_clean = re.findall(custom_alp, str(text_var))
        text_result_var = "".join(text_clean)
        text_result_list.append(text_result_var)
    text_join = " ".join(text_result_list)
    text_result = str.lower(text_join)
    return text_result

In [237]:
def convert_to_float(value):
    '''This function convert string numeric value to float type. Like as 1.200,15
    convert_to_float(value): value is a string numeric value
    '''
    value = value.replace(".","").replace(",",".")
    return float(value)

In [238]:
def whitespace_del(text):
    '''This function provides removing left right white space.\n
    whitespace_del(text): text is a string value as word or sentence.
    '''
    text = text.strip()  # rstrip lstrip
    return text

#### Report

In [239]:
df_entegra = pd.read_excel(r"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Data\Entegra\Test\Entegra Sipariş Listesi.xlsx")
df_entegra

,id,order_number,platform_reference_no,datetime,date_add,entegration,supplier,status,status_name,invoice_country,...,cargo_company,cargo_packet_quantity,model,product_name,store_product_name,invoice_name,payment_type,total_product,PazaryerindenGelenOdemeTutar,total_product_quantity
0,177632,2275858955,9347410816,2024-07-09 13:12:40,2024-07-09 13:13:07,trendyol,trendyol,2,Onaylandı,NaN,...,trendyolekspress,1,004360,Köpek Anti Pire Kene Önleyici Tasma- Pire Önle...,Köpek Anti Pire Kene Önleyici Tasma- Pire Önle...,Köpek Anti Pire Kene Önleyici Tasma- Pire Önle...,Trendyol,1,0,1
1,177633,2275855595,9347405864,2024-07-09 13:10:42,2024-07-09 13:13:10,trendyol,trendyol,2,Onaylandı,NaN,...,trendyolekspress,1,003975,Ib58330 Explorer Şişme Bot 200 95 Kg 185x94x41...,İntex Expoler 200 Şişme Bot Deniz Havuz Göl Bo...,İntex Expoler 200 Şişme Bot Deniz Havuz Göl Bo...,Trendyol,1,0,1
2,177634,2275852647,9347405327,2024-07-09 13:10:29,2024-07-09 13:13:15,trendyol,trendyol,2,Onaylandı,NaN,...,trendyolekspress,1,001603,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...","Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...","Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",Trendyol,1,0,1
3,177635,209253154635-1,209253154635,2024-07-09 13:09:00,2024-07-09 13:14:05,N11,n11,1,Yeni Siparis,NaN,...,mng,1,003628,Koruyucu Gümüş Göğüs Ucu Kapakları Large,Koruyucu Gümüş Göğüs Ucu Kapakları Large,Koruyucu Gümüş Göğüs Ucu Kapakları Large,N11,2,0,2
4,177631,2275848409,9347395851,2024-07-09 13:06:45,2024-07-09 13:08:40,trendyol,trendyol,2,Onaylandı,NaN,...,trendyolekspress,1,002583,"Portatif Katlanabilir Masa, Piknik Masası, Kam...","Portatif Katlanabilir Masa, Piknik Masası, Kam...","Portatif Katlanabilir Masa, Piknik Masası, Kam...",Trendyol,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,177046,2273734443,9345134719,2024-07-08 12:42:50,2024-07-08 12:48:16,trendyol,trendyol,9,İade-İptal,NaN,...,trendyolekspress,1,004442,Bakır Görünümlü Çelik Sahan Seti Omlet Yumurta...,Bakır Görünümlü Çelik Sahan Seti Omlet Yumurta...,Bakır Görünümlü Çelik Sahan Seti Omlet Yumurta...,Trendyol,1,0,1
175,177059,402-7962016-7957952,402-7962016-7957952,2024-07-08 09:50:52,2024-07-08 13:22:14,Amazon,amazon,1,Yeni Siparis,NaN,...,NaN,1,003993,Intex AP635 Quickfill Usb Hava Pompası IP66635,"Intex Mini Quickfill USB hava pompası, hava ak...",Intex AP635 Quickfill Usb Hava Pompası IP66635,Amazon,1,0,1
176,176936,2273279741,9344693452,2024-07-08 06:59:22,2024-07-08 07:01:29,trendyol,trendyol,9,İade-İptal,NaN,...,trendyolekspress,1,004198,Intex Onarım Yaması- Onarım Kiti 6 Pcs IT59631,Tamir Onarım Yaması 3 Adet ( 18'li ) Şişme Bot...,Intex Onarım Yaması- Onarım Kiti 6 Pcs IT59631,Trendyol,1,0,3
177,176935,2273277096,9344692471,2024-07-08 06:52:54,2024-07-08 06:55:59,trendyol,trendyol,9,İade-İptal,NaN,...,trendyolekspress,1,002583,"Portatif Katlanabilir Masa, Piknik Masası, Kam...","Portatif Katlanabilir Masa, Piknik Masası, Kam...","Portatif Katlanabilir Masa, Piknik Masası, Kam...",Trendyol,1,0,1


In [240]:
df_entegra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   id                            179 non-null    int64         
 1   order_number                  179 non-null    object        
 2   platform_reference_no         179 non-null    object        
 3   datetime                      179 non-null    datetime64[ns]
 4   date_add                      179 non-null    datetime64[ns]
 5   entegration                   179 non-null    object        
 6   supplier                      179 non-null    object        
 7   status                        179 non-null    int64         
 8   status_name                   179 non-null    object        
 9   invoice_country               2 non-null      object        
 10  invoice_city                  179 non-null    object        
 11  total                         17

In [241]:
df_entegra.columns = [x.lower() for x in df_entegra.columns]
df_entegra

,id,order_number,platform_reference_no,datetime,date_add,entegration,supplier,status,status_name,invoice_country,...,cargo_company,cargo_packet_quantity,model,product_name,store_product_name,invoice_name,payment_type,total_product,pazaryerindengelenodemetutar,total_product_quantity
0,177632,2275858955,9347410816,2024-07-09 13:12:40,2024-07-09 13:13:07,trendyol,trendyol,2,Onaylandı,NaN,...,trendyolekspress,1,004360,Köpek Anti Pire Kene Önleyici Tasma- Pire Önle...,Köpek Anti Pire Kene Önleyici Tasma- Pire Önle...,Köpek Anti Pire Kene Önleyici Tasma- Pire Önle...,Trendyol,1,0,1
1,177633,2275855595,9347405864,2024-07-09 13:10:42,2024-07-09 13:13:10,trendyol,trendyol,2,Onaylandı,NaN,...,trendyolekspress,1,003975,Ib58330 Explorer Şişme Bot 200 95 Kg 185x94x41...,İntex Expoler 200 Şişme Bot Deniz Havuz Göl Bo...,İntex Expoler 200 Şişme Bot Deniz Havuz Göl Bo...,Trendyol,1,0,1
2,177634,2275852647,9347405327,2024-07-09 13:10:29,2024-07-09 13:13:15,trendyol,trendyol,2,Onaylandı,NaN,...,trendyolekspress,1,001603,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...","Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...","Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",Trendyol,1,0,1
3,177635,209253154635-1,209253154635,2024-07-09 13:09:00,2024-07-09 13:14:05,N11,n11,1,Yeni Siparis,NaN,...,mng,1,003628,Koruyucu Gümüş Göğüs Ucu Kapakları Large,Koruyucu Gümüş Göğüs Ucu Kapakları Large,Koruyucu Gümüş Göğüs Ucu Kapakları Large,N11,2,0,2
4,177631,2275848409,9347395851,2024-07-09 13:06:45,2024-07-09 13:08:40,trendyol,trendyol,2,Onaylandı,NaN,...,trendyolekspress,1,002583,"Portatif Katlanabilir Masa, Piknik Masası, Kam...","Portatif Katlanabilir Masa, Piknik Masası, Kam...","Portatif Katlanabilir Masa, Piknik Masası, Kam...",Trendyol,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,177046,2273734443,9345134719,2024-07-08 12:42:50,2024-07-08 12:48:16,trendyol,trendyol,9,İade-İptal,NaN,...,trendyolekspress,1,004442,Bakır Görünümlü Çelik Sahan Seti Omlet Yumurta...,Bakır Görünümlü Çelik Sahan Seti Omlet Yumurta...,Bakır Görünümlü Çelik Sahan Seti Omlet Yumurta...,Trendyol,1,0,1
175,177059,402-7962016-7957952,402-7962016-7957952,2024-07-08 09:50:52,2024-07-08 13:22:14,Amazon,amazon,1,Yeni Siparis,NaN,...,NaN,1,003993,Intex AP635 Quickfill Usb Hava Pompası IP66635,"Intex Mini Quickfill USB hava pompası, hava ak...",Intex AP635 Quickfill Usb Hava Pompası IP66635,Amazon,1,0,1
176,176936,2273279741,9344693452,2024-07-08 06:59:22,2024-07-08 07:01:29,trendyol,trendyol,9,İade-İptal,NaN,...,trendyolekspress,1,004198,Intex Onarım Yaması- Onarım Kiti 6 Pcs IT59631,Tamir Onarım Yaması 3 Adet ( 18'li ) Şişme Bot...,Intex Onarım Yaması- Onarım Kiti 6 Pcs IT59631,Trendyol,1,0,3
177,176935,2273277096,9344692471,2024-07-08 06:52:54,2024-07-08 06:55:59,trendyol,trendyol,9,İade-İptal,NaN,...,trendyolekspress,1,002583,"Portatif Katlanabilir Masa, Piknik Masası, Kam...","Portatif Katlanabilir Masa, Piknik Masası, Kam...","Portatif Katlanabilir Masa, Piknik Masası, Kam...",Trendyol,1,0,1


In [242]:
df_entegra.columns

Index(['id', 'order_number', 'platform_reference_no', 'datetime', 'date_add',
       'entegration', 'supplier', 'status', 'status_name', 'invoice_country',
       'invoice_city', 'total', 'tax', 'grand_total', 'store_order_status',
       'store_order_status_name', 'note', 'cargo_company',
       'cargo_packet_quantity', 'model', 'product_name', 'store_product_name',
       'invoice_name', 'payment_type', 'total_product',
       'pazaryerindengelenodemetutar', 'total_product_quantity'],
      dtype='object')

In [243]:
df_entegra_select = df_entegra[["model","product_name","datetime","total_product_quantity","entegration",\
    "invoice_city","invoice_country","total","tax","grand_total","cargo_company","status_name"]]
df_entegra_select

,model,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,cargo_company,status_name
0,004360,Köpek Anti Pire Kene Önleyici Tasma- Pire Önle...,2024-07-09 13:12:40,1,trendyol,İzmir,NaN,150.00,30.00,180.00,trendyolekspress,Onaylandı
1,003975,Ib58330 Explorer Şişme Bot 200 95 Kg 185x94x41...,2024-07-09 13:10:42,1,trendyol,Rize,NaN,458.33,91.67,550.00,trendyolekspress,Onaylandı
2,001603,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",2024-07-09 13:10:29,1,trendyol,Nevşehir,NaN,154.17,30.83,185.00,trendyolekspress,Onaylandı
3,003628,Koruyucu Gümüş Göğüs Ucu Kapakları Large,2024-07-09 13:09:00,2,N11,Tekirdağ,NaN,547.28,2.71,549.99,mng,Yeni Siparis
4,002583,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-07-09 13:06:45,2,trendyol,Ankara,NaN,194.40,19.44,213.84,trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...
174,004442,Bakır Görünümlü Çelik Sahan Seti Omlet Yumurta...,2024-07-08 12:42:50,1,trendyol,Kocaeli,NaN,416.67,83.33,500.00,trendyolekspress,İade-İptal
175,003993,Intex AP635 Quickfill Usb Hava Pompası IP66635,2024-07-08 09:50:52,1,Amazon,İstanbul,NaN,390.83,78.17,469.00,NaN,Yeni Siparis
176,004198,Intex Onarım Yaması- Onarım Kiti 6 Pcs IT59631,2024-07-08 06:59:22,3,trendyol,Balıkesir,NaN,158.33,31.67,190.00,trendyolekspress,İade-İptal
177,002583,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-07-08 06:52:54,1,trendyol,Muğla,NaN,195.45,19.55,215.00,trendyolekspress,İade-İptal


In [244]:
df_entegra_select["invoice_country"].unique()

array([nan, 'Azerbaycan', 'Türkiye'], dtype=object)

In [245]:
df_entegra_select["cargo_company"].unique()

array(['trendyolekspress', 'mng', 'hepsijet', nan, 'yurtici', 'eptt'],
      dtype=object)

In [246]:
df_entegra_select["invoice_country"] = df_entegra_select["invoice_country"].fillna("Türkiye")
df_entegra_select

C:\Users\user\AppData\Local\Temp\ipykernel_5836\3182738699.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entegra_select["invoice_country"] = df_entegra_select["invoice_country"].fillna("Türkiye")


,model,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,cargo_company,status_name
0,004360,Köpek Anti Pire Kene Önleyici Tasma- Pire Önle...,2024-07-09 13:12:40,1,trendyol,İzmir,Türkiye,150.00,30.00,180.00,trendyolekspress,Onaylandı
1,003975,Ib58330 Explorer Şişme Bot 200 95 Kg 185x94x41...,2024-07-09 13:10:42,1,trendyol,Rize,Türkiye,458.33,91.67,550.00,trendyolekspress,Onaylandı
2,001603,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",2024-07-09 13:10:29,1,trendyol,Nevşehir,Türkiye,154.17,30.83,185.00,trendyolekspress,Onaylandı
3,003628,Koruyucu Gümüş Göğüs Ucu Kapakları Large,2024-07-09 13:09:00,2,N11,Tekirdağ,Türkiye,547.28,2.71,549.99,mng,Yeni Siparis
4,002583,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-07-09 13:06:45,2,trendyol,Ankara,Türkiye,194.40,19.44,213.84,trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...
174,004442,Bakır Görünümlü Çelik Sahan Seti Omlet Yumurta...,2024-07-08 12:42:50,1,trendyol,Kocaeli,Türkiye,416.67,83.33,500.00,trendyolekspress,İade-İptal
175,003993,Intex AP635 Quickfill Usb Hava Pompası IP66635,2024-07-08 09:50:52,1,Amazon,İstanbul,Türkiye,390.83,78.17,469.00,NaN,Yeni Siparis
176,004198,Intex Onarım Yaması- Onarım Kiti 6 Pcs IT59631,2024-07-08 06:59:22,3,trendyol,Balıkesir,Türkiye,158.33,31.67,190.00,trendyolekspress,İade-İptal
177,002583,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-07-08 06:52:54,1,trendyol,Muğla,Türkiye,195.45,19.55,215.00,trendyolekspress,İade-İptal


In [247]:
df_entegra_select["cargo_company"] = df_entegra_select["cargo_company"].fillna("diger")
df_entegra_select

C:\Users\user\AppData\Local\Temp\ipykernel_5836\474556960.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entegra_select["cargo_company"] = df_entegra_select["cargo_company"].fillna("diger")


,model,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,cargo_company,status_name
0,004360,Köpek Anti Pire Kene Önleyici Tasma- Pire Önle...,2024-07-09 13:12:40,1,trendyol,İzmir,Türkiye,150.00,30.00,180.00,trendyolekspress,Onaylandı
1,003975,Ib58330 Explorer Şişme Bot 200 95 Kg 185x94x41...,2024-07-09 13:10:42,1,trendyol,Rize,Türkiye,458.33,91.67,550.00,trendyolekspress,Onaylandı
2,001603,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",2024-07-09 13:10:29,1,trendyol,Nevşehir,Türkiye,154.17,30.83,185.00,trendyolekspress,Onaylandı
3,003628,Koruyucu Gümüş Göğüs Ucu Kapakları Large,2024-07-09 13:09:00,2,N11,Tekirdağ,Türkiye,547.28,2.71,549.99,mng,Yeni Siparis
4,002583,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-07-09 13:06:45,2,trendyol,Ankara,Türkiye,194.40,19.44,213.84,trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...
174,004442,Bakır Görünümlü Çelik Sahan Seti Omlet Yumurta...,2024-07-08 12:42:50,1,trendyol,Kocaeli,Türkiye,416.67,83.33,500.00,trendyolekspress,İade-İptal
175,003993,Intex AP635 Quickfill Usb Hava Pompası IP66635,2024-07-08 09:50:52,1,Amazon,İstanbul,Türkiye,390.83,78.17,469.00,diger,Yeni Siparis
176,004198,Intex Onarım Yaması- Onarım Kiti 6 Pcs IT59631,2024-07-08 06:59:22,3,trendyol,Balıkesir,Türkiye,158.33,31.67,190.00,trendyolekspress,İade-İptal
177,002583,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-07-08 06:52:54,1,trendyol,Muğla,Türkiye,195.45,19.55,215.00,trendyolekspress,İade-İptal


In [248]:
df_entegra_select.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   model                   179 non-null    object        
 1   product_name            179 non-null    object        
 2   datetime                179 non-null    datetime64[ns]
 3   total_product_quantity  179 non-null    int64         
 4   entegration             179 non-null    object        
 5   invoice_city            179 non-null    object        
 6   invoice_country         179 non-null    object        
 7   total                   179 non-null    float64       
 8   tax                     179 non-null    float64       
 9   grand_total             179 non-null    float64       
 10  cargo_company           179 non-null    object        
 11  status_name             179 non-null    object        
dtypes: datetime64[ns](1), float64(3), int64(1), object

In [249]:
object_columns = df_entegra_select.select_dtypes(include='object')
object_columns

,model,product_name,entegration,invoice_city,invoice_country,cargo_company,status_name
0,004360,Köpek Anti Pire Kene Önleyici Tasma- Pire Önle...,trendyol,İzmir,Türkiye,trendyolekspress,Onaylandı
1,003975,Ib58330 Explorer Şişme Bot 200 95 Kg 185x94x41...,trendyol,Rize,Türkiye,trendyolekspress,Onaylandı
2,001603,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",trendyol,Nevşehir,Türkiye,trendyolekspress,Onaylandı
3,003628,Koruyucu Gümüş Göğüs Ucu Kapakları Large,N11,Tekirdağ,Türkiye,mng,Yeni Siparis
4,002583,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",trendyol,Ankara,Türkiye,trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...
174,004442,Bakır Görünümlü Çelik Sahan Seti Omlet Yumurta...,trendyol,Kocaeli,Türkiye,trendyolekspress,İade-İptal
175,003993,Intex AP635 Quickfill Usb Hava Pompası IP66635,Amazon,İstanbul,Türkiye,diger,Yeni Siparis
176,004198,Intex Onarım Yaması- Onarım Kiti 6 Pcs IT59631,trendyol,Balıkesir,Türkiye,trendyolekspress,İade-İptal
177,002583,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",trendyol,Muğla,Türkiye,trendyolekspress,İade-İptal


In [250]:
object_columns.columns

Index(['model', 'product_name', 'entegration', 'invoice_city',
       'invoice_country', 'cargo_company', 'status_name'],
      dtype='object')

In [251]:
for column in object_columns.columns:
    df_entegra_select[f"{column}"] = df_entegra_select[f"{column}"].apply(lambda x: whitespace_del(x))
    df_entegra_select[f"{column}"] = df_entegra_select[f"{column}"].apply(lambda x: title_func(x))
    
df_entegra_select.head(10)

C:\Users\user\AppData\Local\Temp\ipykernel_5836\327789667.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entegra_select[f"{column}"] = df_entegra_select[f"{column}"].apply(lambda x: whitespace_del(x))
C:\Users\user\AppData\Local\Temp\ipykernel_5836\327789667.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entegra_select[f"{column}"] = df_entegra_select[f"{column}"].apply(lambda x: title_func(x))


,model,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,cargo_company,status_name
0,004360,Köpek Anti Pire Kene Önleyici Tasma- Pire Önle...,2024-07-09 13:12:40,1,Trendyol,İzmir,Türkiye,150.0000,30.0000,180.00,Trendyolekspress,Onaylandı
1,003975,Ib58330 Explorer Şişme Bot 200 95 Kg 185X94X41...,2024-07-09 13:10:42,1,Trendyol,Rize,Türkiye,458.3300,91.6700,550.00,Trendyolekspress,Onaylandı
2,001603,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",2024-07-09 13:10:29,1,Trendyol,Nevşehir,Türkiye,154.1700,30.8300,185.00,Trendyolekspress,Onaylandı
3,003628,Koruyucu Gümüş Göğüs Ucu Kapakları Large,2024-07-09 13:09:00,2,N11,Tekirdağ,Türkiye,547.2800,2.7100,549.99,Mng,Yeni Siparis
4,002583,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-07-09 13:06:45,2,Trendyol,Ankara,Türkiye,194.4000,19.4400,213.84,Trendyolekspress,Onaylandı
5,002921,Baby Yuvarlak Torbalı Oyuncak/Kirli Çamaşır Se...,2024-07-09 13:01:06,2,Trendyol,İstanbul,Türkiye,207.2700,20.7300,228.00,Trendyolekspress,Onaylandı
6,002529,"Kedi Köpek Hamağı, Kedi Köpek Evi, Kedi Köpek ...",2024-07-09 13:00:13,1,Trendyol,Kocaeli,Türkiye,165.8300,33.1700,199.00,Trendyolekspress,Onaylandı
7,003724,Awox Teaplus Mürdüm 2000 W Cam Demlikli Çay Ma...,2024-07-09 12:59:09,1,Trendyol,Mardin,Türkiye,470.8300,94.1700,565.00,Mng,Onaylandı
8,002583,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-07-09 12:58:08,1,Hepsiburada,İstanbul,Türkiye,170.9091,17.0909,188.00,Hepsijet,Onaylandı
9,002508,"Procli̇s Lavanta Yağı, %100 Doğal Uçucu Yağ, L...",2024-07-09 12:58:05,2,Hepsiburada,İstanbul,Türkiye,226.1667,45.2333,271.40,Hepsijet,Onaylandı


In [217]:
# df_entegra_select[object_columns.columns] = object_columns.apply(lambda x: x.str.lower())
# df_entegra_select

In [218]:
# selected_column = ["product_name","entegration","invoice_city","invoice_country","cargo_company","cargo_company","status_name"] 
# for column in selected_column:
#     df_entegra_select[f"{column}"] = df_entegra_select[f"{column}"].apply(lambda x: whitespace_del(x))
#     
# df_entegra_select.head(10)

In [219]:
# selected_column2 = ["entegration","invoice_city","invoice_country","cargo_company","cargo_company","status_name"] 
# for column in selected_column2:
#     df_entegra_select[f"{column}"] = df_entegra_select[f"{column}"].apply(lambda x: capitalize_func(x))
# 
# df_entegra_select.head(10)

In [220]:
df_entegra_select_product = df_entegra_select[["model","product_name"]]
df_entegra_select_product

,model,product_name
0,004360,Köpek Anti Pire Kene Önleyici Tasma- Pire Önle...
1,003975,Ib58330 Explorer Şişme Bot 200 95 Kg 185X94X41...
2,001603,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ..."
3,003628,Koruyucu Gümüş Göğüs Ucu Kapakları Large
4,002583,"Portatif Katlanabilir Masa, Piknik Masası, Kam..."
...,...,...
174,004442,Bakır Görünümlü Çelik Sahan Seti Omlet Yumurta...
175,003993,Intex Ap635 Quickfill Usb Hava Pompası Ip66635
176,004198,Intex Onarım Yaması- Onarım Kiti 6 Pcs It59631
177,002583,"Portatif Katlanabilir Masa, Piknik Masası, Kam..."


In [221]:
df_entegra_select_product["model"].nunique()

63

In [222]:
df_entegra_select_product["product_name"].nunique()

63

In [223]:
df_entegra_select_product = df_entegra_select_product.drop_duplicates()
df_entegra_select_product

,model,product_name
0,004360,Köpek Anti Pire Kene Önleyici Tasma- Pire Önle...
1,003975,Ib58330 Explorer Şişme Bot 200 95 Kg 185X94X41...
2,001603,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ..."
3,003628,Koruyucu Gümüş Göğüs Ucu Kapakları Large
4,002583,"Portatif Katlanabilir Masa, Piknik Masası, Kam..."
...,...,...
166,003910,Aytuğ Erkek Modal 5 Çift Özel Kutulu Patik Çor...
173,002581,"Iksa Devrilmez Ayaklı Portatif Kamp Ocağı, Pik..."
174,004442,Bakır Görünümlü Çelik Sahan Seti Omlet Yumurta...
176,004198,Intex Onarım Yaması- Onarım Kiti 6 Pcs It59631


In [ ]:
df_entegra_select["status_name"].unique()

array(['Onaylandı', 'Yeni Siparis', 'İade-İptal'], dtype=object)

In [267]:
entegration_list_all = list(df_entegra_select["entegration"].unique())
entegration_list_all

['Trendyol', 'N11', 'Hepsiburada', 'Amazon', 'Woocommerce', 'Ciceksepeti']

In [ ]:
df_entegra_select_sales = df_entegra_select[(df_entegra_select["status_name"] == "Onaylandı") | (df_entegra_select["status_name"] == "Yeni Siparis")]
df_entegra_select_sales.reset_index(drop=True, inplace=True)
df_entegra_select_sales

,model,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,cargo_company,status_name
0,004360,Köpek Anti Pire Kene Önleyici Tasma- Pire Önle...,2024-07-09 13:12:40,1,Trendyol,İzmir,Türkiye,150.00,30.00,180.00,Trendyolekspress,Onaylandı
1,003975,Ib58330 Explorer Şişme Bot 200 95 Kg 185X94X41...,2024-07-09 13:10:42,1,Trendyol,Rize,Türkiye,458.33,91.67,550.00,Trendyolekspress,Onaylandı
2,001603,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",2024-07-09 13:10:29,1,Trendyol,Nevşehir,Türkiye,154.17,30.83,185.00,Trendyolekspress,Onaylandı
3,003628,Koruyucu Gümüş Göğüs Ucu Kapakları Large,2024-07-09 13:09:00,2,N11,Tekirdağ,Türkiye,547.28,2.71,549.99,Mng,Yeni Siparis
4,002583,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-07-09 13:06:45,2,Trendyol,Ankara,Türkiye,194.40,19.44,213.84,Trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...
156,002263,"İksa Devrilmez Ayaklı Portatif Kamp Ocağı, Pik...",2024-07-09 00:14:11,1,Trendyol,İstanbul,Türkiye,304.17,60.83,365.00,Eptt,Onaylandı
157,004297,Pirge Maaster Cheef - Ecco Çantalı 5'Li Bıçak ...,2024-07-09 00:09:44,1,Trendyol,Kocaeli,Türkiye,1000.00,200.00,1200.00,Yurtici,Onaylandı
158,003910,Aytuğ Erkek Modal 5 Çift Özel Kutulu Patik Çor...,2024-07-09 00:00:59,1,Trendyol,İstanbul,Türkiye,199.09,19.91,219.00,Trendyolekspress,Onaylandı
159,Tanımsız.,Tanımsız,2024-07-08 23:05:48,1,Amazon,Aydın,Türkiye,1249.17,249.83,1499.00,Diger,Yeni Siparis


In [268]:
entegration_list_sales = list(df_entegra_select_sales["entegration"].unique())
entegration_list_sales

['Trendyol', 'N11', 'Hepsiburada', 'Amazon', 'Woocommerce', 'Ciceksepeti']

In [224]:
df_sales_quantity_entegration = pd.pivot_table(index="model",columns="entegration", values="total_product_quantity", aggfunc='sum', data=df_entegra_select_sales)
df_sales_quantity_entegration = df_sales_quantity_entegration.fillna(0)
df_sales_quantity_entegration.reset_index(inplace=True)
df_sales_quantity_entegration

entegration,model,Amazon,Ciceksepeti,Hepsiburada,N11,Trendyol,Woocommerce
0,001275,0.0,0.0,0.0,0.0,4.0,0.0
1,001284,0.0,0.0,0.0,0.0,7.0,0.0
2,001602,0.0,0.0,0.0,2.0,14.0,0.0
3,001603,0.0,1.0,0.0,0.0,5.0,0.0
4,002243,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...
58,004352,0.0,0.0,0.0,0.0,1.0,0.0
59,004360,0.0,0.0,0.0,0.0,1.0,0.0
60,004382,0.0,0.0,0.0,0.0,3.0,0.0
61,004442,0.0,0.0,0.0,0.0,1.0,0.0


In [225]:
df_sales_quantity_entegration = df_sales_quantity_entegration.rename_axis(None, axis=1)  # del index name
df_sales_quantity_entegration

,model,Amazon,Ciceksepeti,Hepsiburada,N11,Trendyol,Woocommerce
0,001275,0.0,0.0,0.0,0.0,4.0,0.0
1,001284,0.0,0.0,0.0,0.0,7.0,0.0
2,001602,0.0,0.0,0.0,2.0,14.0,0.0
3,001603,0.0,1.0,0.0,0.0,5.0,0.0
4,002243,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...
58,004352,0.0,0.0,0.0,0.0,1.0,0.0
59,004360,0.0,0.0,0.0,0.0,1.0,0.0
60,004382,0.0,0.0,0.0,0.0,3.0,0.0
61,004442,0.0,0.0,0.0,0.0,1.0,0.0


In [152]:
df_sales_quantity_entegration["total_product_sales"] = df_sales_quantity_entegration["Amazon"] + df_sales_quantity_entegration["Ciceksepeti"] + \
    df_sales_quantity_entegration["Hepsiburada"] + df_sales_quantity_entegration["N11"] + df_sales_quantity_entegration["Trendyol"] + df_sales_quantity_entegration["Woocommerce"]
df_sales_quantity_entegration    

,model,Amazon,Ciceksepeti,Hepsiburada,N11,Trendyol,Woocommerce,total_product_sales
0,001275,0.0,0.0,0.0,0.0,4.0,0.0,4.0
1,001284,0.0,0.0,0.0,0.0,7.0,0.0,7.0
2,001602,0.0,0.0,0.0,2.0,14.0,0.0,16.0
3,001603,0.0,1.0,0.0,0.0,5.0,0.0,6.0
4,002243,0.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...
58,004352,0.0,0.0,0.0,0.0,1.0,0.0,1.0
59,004360,0.0,0.0,0.0,0.0,1.0,0.0,1.0
60,004382,0.0,0.0,0.0,0.0,3.0,0.0,3.0
61,004442,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [153]:
df_sales_name_quantity_entegration  = pd.merge(df_sales_quantity_entegration, df_entegra_select_product, how="left", on="model")
df_sales_name_quantity_entegration.drop_duplicates(inplace=True)
df_sales_name_quantity_entegration 

,model,Amazon,Ciceksepeti,Hepsiburada,N11,Trendyol,Woocommerce,total_product_sales,product_name
0,001275,0.0,0.0,0.0,0.0,4.0,0.0,4.0,alas çok amaçlı bavul i̇çi düzenleyici gri çiz...
1,001284,0.0,0.0,0.0,0.0,7.0,0.0,7.0,alas çok amaçlı dörtlü çekmece düzenleyici org...
2,001602,0.0,0.0,0.0,2.0,14.0,0.0,16.0,"ahşap özel i̇sim baskılı, oval kedi köpek mama..."
3,001603,0.0,1.0,0.0,0.0,5.0,0.0,6.0,"ahşap özel i̇sim baskılı, kare kedi köpek mama..."
4,002243,0.0,0.0,0.0,0.0,1.0,0.0,1.0,"brillahome araba koltuk kilimi, koltuk minder,..."
...,...,...,...,...,...,...,...,...,...
58,004352,0.0,0.0,0.0,0.0,1.0,0.0,1.0,awox şarjli di̇key süpürge mavi̇
59,004360,0.0,0.0,0.0,0.0,1.0,0.0,1.0,köpek anti pire kene önleyici tasma- pire önle...
60,004382,0.0,0.0,0.0,0.0,3.0,0.0,3.0,çok amaçlı multi i̇nce temizlik fırçası derz a...
61,004442,0.0,0.0,0.0,0.0,1.0,0.0,1.0,bakır görünümlü çelik sahan seti omlet yumurta...


In [274]:
df_entegra_select["status_name"].unique()

array(['Onaylandı', 'Yeni Siparis', 'İade-İptal'], dtype=object)

In [275]:
df_entegra_select_cancel = df_entegra_select[(df_entegra_select["status_name"] == "İade-İptal")]
df_entegra_select_cancel.reset_index(drop=True, inplace=True)
df_entegra_select_cancel

,model,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,cargo_company,status_name
0,001603,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",2024-07-09 12:51:56,2,Trendyol,İstanbul,Türkiye,279.17,55.83,335.00,Trendyolekspress,İade-İptal
1,004352,Awox Şarjli Di̇key Süpürge Mavi̇,2024-07-09 12:16:42,1,Trendyol,İstanbul,Türkiye,2191.67,438.33,2630.00,Mng,İade-İptal
2,002263,"İksa Devrilmez Ayaklı Portatif Kamp Ocağı, Pik...",2024-07-09 10:29:51,1,Trendyol,Edirne,Türkiye,288.33,57.67,346.00,Mng,İade-İptal
3,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-09 10:00:58,2,Trendyol,İstanbul,Türkiye,196.75,35.09,231.84,Trendyolekspress,İade-İptal
4,004297,Pirge Maaster Cheef - Ecco Çantalı 5'Li Bıçak ...,2024-07-09 09:42:45,1,Trendyol,İstanbul,Türkiye,1000.00,200.00,1200.00,Mng,İade-İptal
5,003977,Ib58332 Explorer Şişme Bot 300 186Kg -Intex,2024-07-09 08:16:29,1,Trendyol,Isparta,Türkiye,958.33,191.67,1150.00,Mng,İade-İptal
6,002583,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-07-09 02:06:27,1,Trendyol,Sakarya,Türkiye,181.82,18.18,200.00,Trendyolekspress,İade-İptal
7,002583,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-07-09 01:01:54,1,Trendyol,Mersin,Türkiye,181.82,18.18,200.00,Trendyolekspress,İade-İptal
8,002263,"İksa Devrilmez Ayaklı Portatif Kamp Ocağı, Pik...",2024-07-08 19:01:23,1,Trendyol,Osmaniye,Türkiye,280.00,56.00,336.00,Mng,İade-İptal
9,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-08 18:13:49,1,Trendyol,Ankara,Türkiye,154.17,30.83,185.00,Trendyolekspress,İade-İptal


In [281]:
entegration_list_all = list(df_entegra_select["entegration"].unique())
entegration_list_all

['Trendyol', 'N11', 'Hepsiburada', 'Amazon', 'Woocommerce', 'Ciceksepeti']

In [276]:
entegration_list_cancel = list(df_entegra_select_cancel["entegration"].unique())
entegration_list_cancel

['Trendyol']

In [277]:
df_cancel_quantity_entegration = pd.pivot_table(index="model",columns="entegration", values="total_product_quantity", aggfunc='sum', data=df_entegra_select_cancel)
df_cancel_quantity_entegration = df_cancel_quantity_entegration.fillna(0)
df_cancel_quantity_entegration.reset_index(inplace=True)
df_cancel_quantity_entegration

entegration,model,Trendyol
0,001602,3
1,001603,2
2,002263,5
3,002581,1
4,002583,3
5,002941,3
6,003977,1
7,004198,3
8,004297,1
9,004316,1


In [278]:
df_cancel_quantity_entegration = df_cancel_quantity_entegration.rename_axis(None, axis=1)  # del index name
df_cancel_quantity_entegration

,model,Trendyol
0,001602,3
1,001603,2
2,002263,5
3,002581,1
4,002583,3
5,002941,3
6,003977,1
7,004198,3
8,004297,1
9,004316,1


In [282]:
for entegration in entegration_list_all:
    if entegration in entegration_list_cancel:
        df_cancel_quantity_entegration[f"{entegration}"] = 0
    else:
        pass

In [283]:
# df_cancel_quantity_entegration["Hepsiburada"] = 0
# df_cancel_quantity_entegration["Trendyol"] = 0

In [284]:
df_cancel_quantity_entegration["total_product_cancel"] = df_cancel_quantity_entegration["Amazon"] + df_cancel_quantity_entegration["Ciceksepeti"] + \
    df_cancel_quantity_entegration["Hepsiburada"] + df_cancel_quantity_entegration["N11"] + df_cancel_quantity_entegration["Trendyol"] + df_cancel_quantity_entegration["Woocommerce"]
df_cancel_quantity_entegration   

KeyError: 'Amazon'

In [ ]:
df_cancel_name_quantity_entegration  = pd.merge(df_cancel_quantity_entegration, df_entegra_select_product, how="left", on="model")
df_cancel_name_quantity_entegration.drop_duplicates(inplace=True)
df_cancel_name_quantity_entegration 

,model,Amazon,Ciceksepeti,Hepsiburada,N11,Trendyol,Woocommerce,total_product_sales,product_name
0,001275,0.0,0.0,0.0,0.0,4.0,0.0,4.0,alas çok amaçlı bavul i̇çi düzenleyici gri çiz...
1,001284,0.0,0.0,0.0,0.0,7.0,0.0,7.0,alas çok amaçlı dörtlü çekmece düzenleyici org...
2,001602,0.0,0.0,0.0,2.0,14.0,0.0,16.0,"ahşap özel i̇sim baskılı, oval kedi köpek mama..."
3,001603,0.0,1.0,0.0,0.0,5.0,0.0,6.0,"ahşap özel i̇sim baskılı, kare kedi köpek mama..."
4,002243,0.0,0.0,0.0,0.0,1.0,0.0,1.0,"brillahome araba koltuk kilimi, koltuk minder,..."
...,...,...,...,...,...,...,...,...,...
58,004352,0.0,0.0,0.0,0.0,1.0,0.0,1.0,awox şarjli di̇key süpürge mavi̇
59,004360,0.0,0.0,0.0,0.0,1.0,0.0,1.0,köpek anti pire kene önleyici tasma- pire önle...
60,004382,0.0,0.0,0.0,0.0,3.0,0.0,3.0,çok amaçlı multi i̇nce temizlik fırçası derz a...
61,004442,0.0,0.0,0.0,0.0,1.0,0.0,1.0,bakır görünümlü çelik sahan seti omlet yumurta...
